In [1]:
pip install llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.6/917.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [ ]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_KEY


In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

openai_api_key="API_KEY"
llm = ChatOpenAI(openai_api_key="OPENAI_API_KEY")

# chat(
#     [
#         HumanMessage(
#             content="Give me a specific activity suggestion based on my passion of programming. Your response length is one sentence maximum."
#         )
#     ]
# )

In [ ]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
####
from langchain.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='/Users/danielhoskins/src/llms/llms_project_GrocerEase/grocery.csv', source_column="Description")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=2000, memory_key="chat_history", return_messages=True
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

retriever = vectorstore.as_retriever()

In [ ]:
# NEw code
# from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
qa_system_template = """You are a helpful AI assistant focused solely on recommending grocery items to online shoppers. Your function is to suggest food and related consumables that can be found in a grocery store. No matter the context or the nature of the user's question, your responses should always involve grocery item recommendations. If a user inquires about needs for a specific situation or event, you will only recommend grocery items that are suitable for that context.
# ----------------
# {context}"""

messages = [
    SystemMessagePromptTemplate.from_template(qa_system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]
qa_system_prompt = ChatPromptTemplate.from_messages(messages)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, condense_question_prompt=qa_system_prompt,
                                            verbose=True, combine_docs_chain_kwargs={"prompt": qa_system_prompt})

In [ ]:
qa({"question": "I'm going on a 3-day backpacking trip through the smokey mountains. What will I need?"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI assistant focused solely on recommending grocery items to online shoppers. Your function is to suggest food and related consumables that can be found in a grocery store. No matter the context or the nature of the user's question, your responses should always involve grocery item recommendations. If a user inquires about needs for a specific situation or event, you will only recommend grocery items that are suitable for that context. 
# ----------------
# ﻿SKU: Nature Valley granola bars
Type: Snacks
Cost: $2.99
Quantity (g): 170
Description: Oat-based snack bars

﻿SKU: Triscuits
Type: Snacks
Cost: $3.49
Quantity (g): 170
Description: Wheat cracker snacks

﻿SKU: Oreos
Type: Snacks
Cost: $3.49
Quantity (g): 170
Description: Chocolate sandwich cookies

﻿SKU: Skippy peanut butter
Type: Spreads
Cost: $3.29
Quantity (g): 700
Description: Creamy or chunky spre

{'question': "I'm going on a 3-day backpacking trip through the smokey mountains. What will I need?",
 'chat_history': [HumanMessage(content="I'm going on a 3-day backpacking trip through the smokey mountains. What will I need?"),
  AIMessage(content="For your 3-day backpacking trip through the Smoky Mountains, here are some grocery items you might find useful:\n\n1. Nature Valley granola bars: These oat-based snack bars are convenient and provide energy for your hikes.\n2. Triscuits: These wheat cracker snacks can be a great source of carbohydrates and are easy to pack.\n3. Skippy peanut butter: A jar of Skippy peanut butter can be a versatile spread for sandwiches or as a source of protein and healthy fats.\n4. Bottled water: Staying hydrated is crucial during outdoor activities, so make sure to bring enough water for your trip.\n5. Dried fruit and nuts: These lightweight and nutritious snacks can provide you with a quick energy boost.\n6. Instant oatmeal: Easy to prepare and a great

In [ ]:
memory.clear()
qa({"question": "I'm having a halloween party. Recommend some items for me"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI assistant focused solely on recommending grocery items to online shoppers. Your function is to suggest food and related consumables that can be found in a grocery store. No matter the context or the nature of the user's question, your responses should always involve grocery item recommendations. If a user inquires about needs for a specific situation or event, you will only recommend grocery items that are suitable for that context. 
# ----------------
# ﻿SKU: Spooky Cookies
Type: Snacks
Cost: 4.99
Quantity (g): 12
Description: Halloween themed cookies

﻿SKU: Haunted chips
Type: Snacks
Cost: $5.99
Quantity (g): 6
Description: Scary and festive treats perfect for the fall season

﻿SKU: Keebler cookies
Type: Snacks
Cost: $2.99
Quantity (g): 170
Description: Assorted cookies

﻿SKU: Chips Ahoy cookies
Type: Snacks
Cost: $2.99
Quantity (g): 170
Description: 

{'question': "I'm having a halloween party. Recommend some items for me",
 'chat_history': [HumanMessage(content="I'm having a halloween party. Recommend some items for me"),
  AIMessage(content="For your Halloween party, I recommend the following items:\n\n1. Spooky Cookies: These Halloween-themed cookies will be a hit with your guests. They are not only delicious but also add a festive touch to your party.\n\n2. Haunted chips: These scary and festive treats are perfect for the fall season. They will add a spooky element to your party and satisfy your guests' snack cravings.\n\n3. Keebler cookies: These assorted cookies are a crowd-pleaser. With a variety of flavors, they are sure to please everyone at your party.\n\n4. Chips Ahoy cookies: These classic chocolate chip cookies are always a hit. They are a timeless treat that your guests will enjoy.\n\nWith these snacks, you'll have a great variety of treats for your Halloween party. Enjoy your spooky celebration!")],
 'answer': "For yo